# Chaînes de Markov : page Rank de Google

## Étape 1 : Importer les bibliothèques nécessaires

In [1]:
import numpy as np

La bibliothèque numpy permet de manipuler des matrices et des vecteurs efficacement en Python.

## Étape 2 : Définir la matrice d'adjacence A

La matrice d'adjacence `A` représente les connexions entre les pages web. Si une page `i` pointe vers une page `j`, alors `A[i][j] = 1`. Sinon, `A[i][j] = 0`.

![drawing](./image.png)


In [3]:
# Définition de la matrice d'adjacence A
A = np.array([
    [1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0]
])

## Étape 3 : Calculer la matrice de probabilité P

Pour que les valeurs dans chaque ligne de la matrice représentent des probabilités, on divise chaque élément de la ligne par le nombre de liens sortants (ou connexions) de la page correspondante.

In [4]:
# Calcul de la matrice de probabilité P
P = np.zeros_like(A, dtype=float)  # Matrice P de la même taille que A avec des 0
for i in range(A.shape[0]):
    liens = np.sum(A[i, :])  # Somme des éléments de la ligne i
    if liens > 0:  # Evite la division par zéro
        P[i, :] = A[i, :] / liens

Explication :

- Pour chaque ligne `i` dans `A`, on calcule le nombre de liens sortants.
- Si une page a au moins un lien sortant, chaque élément de la ligne est divisé par ce nombre.
- Ainsi, chaque ligne de `P` contient des valeurs de probabilité qui représentent les chances de suivre chaque lien sortant de la page `i`.

## Étape 4 : Initialiser le vecteur X0

Le vecteur $ X_0 $ est notre vecteur d'état initial, qui représente la probabilité d'être sur chaque page au tout début. Ici, on initialise $ X_0 $ avec la première page ayant une probabilité de 1 et toutes les autres à 0.

In [5]:
# Création du vecteur X0
X0 = np.zeros(A.shape[0])
X0[0] = 1  # On initialise X0 avec 1 dans la première case

## Étape 5 : Calculer le vecteur de probabilité Xn

On itère pour mettre à jour les probabilités jusqu'à convergence (stabilisation des valeurs). À chaque étape, le nouveau vecteur $ X_{n+1} $ est obtenu en multipliant l'état actuel $ X_n $ par la matrice de probabilité `P`.

In [ ]:
# Calcul de Xn
Xn = X0
Xn1 = Xn @ P
while not np.array_equal(Xn1, Xn):  # Boucle jusqu'à convergence
    Xn = Xn1
    Xn1 = Xn @ P

Explication :

- Tant que le vecteur $ X_{n+1} $ diffère de $ X_n $, on met à jour $ X_n $.
- La condition de la boucle `np.array_equal(Xn1, Xn)` permet de vérifier si les probabilités sont stabilisées (pas de changement entre deux itérations).

## Étape 6 : Afficher le classement des pages

Pour afficher le classement des pages, on trie les valeurs de $ X_{n+1} $ en ordre décroissant. Ensuite, on affiche les pages et leurs scores PageRank.

In [8]:
# Affichage des résultats
Vrang = np.sort(Xn1)[::-1]  # Trie des valeurs de Xn en ordre décroissant
rangP = np.argsort(Xn1)[::-1]  # Indices des pages triées

print("Classement des pages web selon PageRank :")
for i in range(len(rangP)):
    print(f"Page {rangP[i] + 1} : PageRank = {Xn1[rangP[i]]}")


Classement des pages web selon PageRank :
Page 5 : PageRank = 0.20869565217391242
Page 7 : PageRank = 0.1478260869565213
Page 9 : PageRank = 0.13913043478260823
Page 1 : PageRank = 0.08695652173913018
Page 6 : PageRank = 0.08695652173913017
Page 8 : PageRank = 0.06956521739130414
Page 12 : PageRank = 0.06956521739130411
Page 11 : PageRank = 0.06956521739130411
Page 10 : PageRank = 0.06956521739130411
Page 4 : PageRank = 0.023188405797101387
Page 3 : PageRank = 0.01739130434782604
Page 2 : PageRank = 0.011594202898550694


Explication :

- `np.sort(Xn1)[::-1]` trie les valeurs en ordre décroissant. <br>
- `np.argsort(Xn1)[::-1]` obtient les indices des pages dans l'ordre décroissant de leur PageRank. <br>
- On utilise une boucle pour afficher chaque page avec son score PageRank.